# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [13]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = 'C:\\Users\\Master\\'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
df.describe()

,Pclass,Age,SibSp,Parch,Fare
count,891.000000,714.000000,891.000000,891.000000,891.000000
mean,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.420000,0.000000,0.000000,0.000000
25%,2.000000,20.125000,0.000000,0.000000,7.910400
50%,3.000000,28.000000,0.000000,0.000000,14.454200
75%,3.000000,38.000000,1.000000,0.000000,31.000000
max,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
df.describe(include='all')

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,"Lovell, Mr. John Hall (""Henry"")",male,NaN,NaN,NaN,CA. 2343,NaN,B96 B98,S
freq,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [10]:
df[df['Fare']==0]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
179,3,"Leonard, Mr. Lionel",male,36.0,0,0,LINE,0.0,NaN,S
263,1,"Harrison, Mr. William",male,40.0,0,0,112059,0.0,B94,S
271,3,"Tornquist, Mr. William Henry",male,25.0,0,0,LINE,0.0,NaN,S
277,2,"Parkes, Mr. Francis ""Frank""",male,NaN,0,0,239853,0.0,NaN,S
302,3,"Johnson, Mr. William Cahoone Jr",male,19.0,0,0,LINE,0.0,NaN,S
413,2,"Cunningham, Mr. Alfred Fleming",male,NaN,0,0,239853,0.0,NaN,S
466,2,"Campbell, Mr. William",male,NaN,0,0,239853,0.0,NaN,S
481,2,"Frost, Mr. Anthony Wood ""Archie""",male,NaN,0,0,239854,0.0,NaN,S
597,3,"Johnson, Mr. Alfred",male,49.0,0,0,LINE,0.0,NaN,S
633,1,"Parr, Mr. William Henry Marsh",male,NaN,0,0,112052,0.0,NaN,S


# 為什麼有些票價0元?

# 可以看到Age跟Cabin有缺失值

In [16]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
mean_df = df.groupby(['Pclass'])['Fare'].mean().reset_index()
mode_df = df.groupby(['Pclass'])['Fare'].apply(lambda x : x.mode()[0]).reset_index()
median_df = df.groupby(['Pclass'])['Fare'].median().reset_index()
max_df = df.groupby(['Pclass'])['Fare'].max().reset_index()
min_df = df.groupby(['Pclass'])['Fare'].min().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on='Pclass')
temp = pd.merge(temp, median_df, how='left', on='Pclass')
temp = pd.merge(temp, max_df, how='left', on='Pclass')
temp = pd.merge(temp, min_df, how='left', on='Pclass')
temp.columns = ['Pclass', 'Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max', 'Fare_Min']
temp

,Pclass,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max,Fare_Min
0,1,84.154687,26.55,60.2875,512.3292,0.0
1,2,20.662183,13.00,14.2500,73.5000,0.0
2,3,13.675550,8.05,8.0500,69.5500,0.0


In [15]:
# 取船票票號(Ticket), 對乘客年齡(Age)做群聚編碼
df['Ticket'] = df['Ticket'].fillna('None')
df['Age'] = df['Age'].fillna(df['Age'].mean())

mean_df = df.groupby(['Ticket'])['Age'].mean().reset_index()
mode_df = df.groupby(['Ticket'])['Age'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['Ticket'])['Age'].median().reset_index()
max_df = df.groupby(['Ticket'])['Age'].max().reset_index()
min_df = df.groupby(['Ticket'])['Age'].min().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['Ticket'])
temp = pd.merge(temp, median_df, how='left', on=['Ticket'])
temp = pd.merge(temp, max_df, how='left', on=['Ticket'])
temp = pd.merge(temp, min_df, how='left', on=['Ticket'])
temp.columns = ['Ticket', 'Age_Mean', 'Age_Mode', 'Age_Median', 'Age_Max', 'Age_Min']
temp.head()

,Ticket,Age_Mean,Age_Mode,Age_Median,Age_Max,Age_Min
0,110152,26.333333,16.000000,30.000000,33.0,16.000000
1,110413,36.333333,18.000000,39.000000,52.0,18.000000
2,110465,38.349559,29.699118,38.349559,47.0,29.699118
3,110564,28.000000,28.000000,28.000000,28.0,28.000000
4,110813,60.000000,60.000000,60.000000,60.0,60.000000


In [17]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


In [18]:
df_test = pd.merge(df, temp, how='left', on=['Pclass'])

In [24]:
df_test.head()

,Pclass,Age,SibSp,Parch,Fare,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max,Fare_Min
0,3,22.0,1,0,7.2500,13.675550,8.05,8.0500,69.5500,0.0
1,1,38.0,1,0,71.2833,84.154687,26.55,60.2875,512.3292,0.0
2,3,26.0,0,0,7.9250,13.675550,8.05,8.0500,69.5500,0.0
3,1,35.0,1,0,53.1000,84.154687,26.55,60.2875,512.3292,0.0
4,3,35.0,0,0,8.0500,13.675550,8.05,8.0500,69.5500,0.0


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [20]:
#原始特徵
df_minus = df_test.drop(['Fare_Mean', 'Fare_Median', 'Fare_Mode', 'Fare_Max', 'Fare_Min'], axis=1)

In [22]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6982266036406296

In [23]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df_test)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6982329516378366